In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import supabase_client
from src.update_pipeline import update_training_week
from src.types.update_pipeline import ExeType
from src import auth_manager
from src.utils import datetime_now_est, get_last_sunday

import os

# New Week Update

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
update_training_week(user, ExeType.NEW_WEEK, dt=get_last_sunday())

# Mid Week Update

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
update_training_week(user, ExeType.MID_WEEK, dt=datetime_now_est())

# gen_training_plan

In [7]:
import os
from pprint import pprint

from src.utils import datetime_now_est
from src import supabase_client, activities, training_plan, auth_manager

In [9]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
strava_client = auth_manager.get_strava_client(user.athlete_id)
daily_activity = activities.get_daily_activity(strava_client, dt=datetime_now_est(), num_weeks=52)
weekly_summaries = activities.get_weekly_summaries(daily_activity=daily_activity)
resp = await training_plan.gen_training_plan(user, weekly_summaries, dt=datetime_now_est())

pprint(resp.training_plan_weeks)

No rates present in response headers


[TrainingPlanWeek(week_start_date=datetime.date(2025, 2, 3), week_number=1, n_weeks_until_race=9, week_type=<WeekType.BUILD: 'build'>, total_distance=32.0, long_run_distance=14.0, notes="This week marks the beginning of your training block as you prepare for the Ultra Marathon on April 13th. With a target volume of 32 miles and a 14-mile long run, this week will lay the foundation to gradually increase your mileage while maintaining a balance between intensity and recovery. By building a strong base, you'll progressively enhance your endurance and stamina, crucial for tackling the demanding distances of an ultra marathon."),
 TrainingPlanWeek(week_start_date=datetime.date(2025, 2, 10), week_number=2, n_weeks_until_race=8, week_type=<WeekType.BUILD: 'build'>, total_distance=35.0, long_run_distance=16.0, notes='This week is designed to gradually increase your running volume, preparing your body for the upcoming demands of ultra-marathon training. By building up to 35 miles and including 

# Observability

In [14]:
import json
import polars as pl

with open("observe.jsonl", "r") as file:
    data = [json.loads(line) for line in file]

df = pl.DataFrame(data).drop([pl.col("messages"), pl.col("content")])

df

generation_name,response_id,model,completion_tokens,prompt_tokens,total_tokens,duration
str,str,str,i64,i64,i64,f64
"""gen_training_plan""","""chatcmpl-AvrEKW37NeViX59mgcut0…","""gpt-4o-2024-08-06""",702,2144,2846,11.671468
"""gen_coaches_notes""","""chatcmpl-AvrEdWZf0CaQkYsEEEtY8…","""gpt-4o-2024-08-06""",38,842,880,1.387268
"""gen_coaches_notes""","""chatcmpl-AvrEdndqvE8lRTjCX0Si5…","""gpt-4o-2024-08-06""",63,842,905,4.322935
"""gen_coaches_notes""","""chatcmpl-AvrEd7yuPuzGjinWJDl4p…","""gpt-4o-2024-08-06""",66,834,900,1.382767
"""gen_coaches_notes""","""chatcmpl-AvrEdHuxZRTJzfyq4lynn…","""gpt-4o-2024-08-06""",69,842,911,4.054727
…,…,…,…,…,…,…
"""gen_coaches_notes""","""chatcmpl-AvrIbtNW1eMiVroZgP7w5…","""gpt-4o-2024-08-06""",56,834,890,1.109765
"""gen_coaches_notes""","""chatcmpl-AvrIbHwjDmeaPt6JL6hWw…","""gpt-4o-2024-08-06""",60,842,902,4.5356
"""gen_coaches_notes""","""chatcmpl-AvrIbb8eL8H8LrHbPZGEZ…","""gpt-4o-2024-08-06""",79,1235,1314,2.680341


In [15]:
df.group_by(pl.col("generation_name")).agg(pl.col('duration').mean())

generation_name,duration
str,f64
"""gen_pseudo_training_week""",1.77377
"""gen_training_plan""",11.659811
"""gen_coaches_notes""",2.847257
"""gen_training_week""",3.088549


In [13]:
df.sum().select(pl.col("duration"))

duration
f64
32.532163
